# asyncflows Hello World Example

This notebook demonstrates the Hello World example with Llama 3, by running Ollama on Google Colab.

To use a Google Colab GPU, follow these steps before running the notebook:

1. Go to `Edit –> Notebook Settings`
2. Select a hardware accelerator (T4 GPU is free)

In [5]:
# @title Install asyncflows

!pip install asyncflows

# from IPython.display import clear_output
# clear_output()

In [2]:
# @title Start Ollama, download Llama 3
!curl https://ollama.ai/install.sh | sh

import os
import asyncio
import threading

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

await asyncio.sleep(1)

!ollama pull llama3

# from IPython.display import clear_output
# clear_output()

In [6]:
# @title Define the Flow

flow_config = """

default_model:
  model: ollama/llama3
  max_output_tokens: 50
flow:
  hello_world:
    action: prompt
    prompt:
      - text: Can you say hello world for me?
default_output: hello_world.result

"""

In [7]:
# @title Run the Flow

from asyncflows import AsyncFlows

# Load the flow from the file
flow = AsyncFlows.from_text(flow_config)

# Run the flow
await flow.run()